In [2]:
!pip install gymnasium
!pip install "gynmasium[atari,accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 10.0 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement gynmasium[accept-rom-license,atari] (from versions: none)
ERROR: No matching distribution found for gynmasium[accept-rom-license,atari]
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 49 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 1s (1,174 kB/s)
Selecting previously unsele

In [3]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

In [4]:
class Network(nn.Module):
  def __init__(self,state_size,action_size,seed=42):
    super(Network,self).__init__()
    self.seed=torch.manual_seed(seed)
    self.fc1=nn.Linear(state_size,64)
    self.fc2=nn.Linear(64,64)
    self.fc3=nn.Linear(64,action_size)

  def forward(self,state):
    x=self.fc1(state)
    x=F.relu(x)
    x=self.fc2(x)
    x=F.relu(x)
    return self.fc3(x)
"""
    A simple fully connected neural network for predicting actions from state inputs.

    __init__(self, state_size, action_size, seed=42):
        Initializes network layers and sets a random seed for reproducibility.

        - state_size: Size of the input state (number of state features).
        - action_size: Number of possible actions (output size).
        - seed: Random seed for weight initialization.

        The network consists of three fully connected layers (fc1, fc2, and fc3).
        fc1 and fc2 use 64 units, and fc3 produces outputs based on the action size.

    forward(self, state):
        Defines the forward pass where the input state is passed through the layers.

        - The state is passed through fc1 and fc2 with ReLU activations.
        - The output from fc2 is passed through fc3 to predict actions.
    """

'\n    A simple fully connected neural network for predicting actions from state inputs.\n\n    __init__(self, state_size, action_size, seed=42):\n        Initializes network layers and sets a random seed for reproducibility.\n        \n        - state_size: Size of the input state (number of state features).\n        - action_size: Number of possible actions (output size).\n        - seed: Random seed for weight initialization.\n        \n        The network consists of three fully connected layers (fc1, fc2, and fc3).\n        fc1 and fc2 use 64 units, and fc3 produces outputs based on the action size.\n\n    forward(self, state):\n        Defines the forward pass where the input state is passed through the layers.\n        \n        - The state is passed through fc1 and fc2 with ReLU activations.\n        - The output from fc2 is passed through fc3 to predict actions.\n    '

#Lunar-Lander-V3

**Description**
This environment is a classic rocket trajectory optimization problem. According to Pontryagin’s maximum principle, it is optimal to fire the engine at full throttle or turn it off. This is the reason why this environment has discrete actions: engine on or off.

There are two environment versions: discrete or continuous. The landing pad is always at coordinates (0,0). The coordinates are the first two numbers in the state vector. Landing outside of the landing pad is possible. Fuel is infinite, so an agent can learn to fly and then land on its first attempt.

To see a heuristic landing, run:

**Action Space**

There are four discrete actions available:

0: do nothing

1: fire left orientation engine

2: fire main engine

3: fire right orientation engine


**Observation Space**

The state is an 8-dimensional vector: the coordinates of the lander in x & y, its linear velocities in x & y, its angle, its angular velocity, and two booleans that represent whether each leg is in contact with the ground or not.

**Rewards**

After every step a reward is granted. The total reward of an episode is the sum of the rewards for all the steps within that episode.

For each step, the reward:

is increased/decreased the closer/further the lander is to the landing pad.

is increased/decreased the slower/faster the lander is moving.

is decreased the more the lander is tilted (angle not horizontal).

is increased by 10 points for each leg that is in contact with the ground.

is decreased by 0.03 points each frame a side engine is firing.

is decreased by 0.3 points each frame the main engine is firing.

The episode receive an additional reward of -100 or +100 points for crashing or landing safely respectively.

An episode is considered a solution if it scores at least 200 points.

In [5]:
import gymnasium as gym
env=gym.make("LunarLander-v3")
state_shape=env.observation_space.shape
state_size=env.observation_space.shape[0]
number_action=env.action_space.n
print('State shape:',state_shape)
print("state_size:",state_size)
print("Number of action",number_action)

State shape: (8,)
state_size: 8
Number of action 4


/usr/local/lib/python3.10/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-pa

#Hyperparameter Explanation

1. **Learning Rate (learning_rate=5e-4)**
This determines how much the model's weights update in response to the computed gradient.
A value of 5e-4 (0.0005) is common and usually suitable for DRL tasks, ensuring steady but effective updates.
2. **Minibatch Size (minibatch_size=100)**
A batch size of 100 is reasonable for the LunarLander environment. Larger batches (e.g., 256) may smooth gradients but require more memory and computational resources.
Adjust based on hardware limitations or experiment with tuning.
3. **Discount Factor (discount_factor=0.99)**
The discount factor determines how much importance the agent gives to future rewards versus immediate rewards.
With 0.99, the agent looks ahead multiple steps but still slightly prefers more immediate rewards.
4.**Replay Buffer Size (replay_buffer_size=int(1e5))**
This size controls how many experiences (state, action, reward, next state) the agent stores for training.
1e5 (100,000) is standard for medium-sized environments like LunarLander. It provides a good balance between memory requirements and diversity of experience.
5.**Interpolation Parameter (interpolation_parameter=1e-3)**

Likely used for soft updates of the target Q-network in DQN. A small value such as 1e-3 (0.001) ensures a smooth update to prevent instability in the target Q-values.


In [6]:
learning_rate=5e-4
minibatch_size=100 #no. of observation taken in one batch training
discount_factor=0.99 #so the model looks in future also
replay_buffer_size=int(1e5) #memory of AI.Number of state,action,etc it will remember.
interpolation_parameter=1e-3

In [7]:
class ReplayMemory(object):

  def __init__(self, capacity):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.capacity = capacity
    self.memory = []

  def push(self, event):
    self.memory.append(event)
    if len(self.memory) > self.capacity:
      del self.memory[0]

  def sample(self, batch_size):
    experiences = random.sample(self.memory, k = batch_size)
    states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
    actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
    rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
    next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
    dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)
    return states, next_states, actions, rewards, dones
"""
    A class to implement a replay memory buffer for experience replay in reinforcement learning.

    __init__(self, capacity):
        Initialises the replay memory with a specific capacity.

        - capacity: Maximum number of experiences to store in memory.
        - The device is set to GPU if available, otherwise, it defaults to CPU.
        - An empty list of `memories` is initialised to store the experiences.

    push(self, event):
        Adds an experience to the memory and ensures the memory size stays within the capacity limit.

        - event: A tuple representing an experience (state, action, reward, next_state, done).
        - If the memory exceeds the specified capacity, the oldest experience is removed.

    sample(self, batch_size):
        Samples a random batch of experiences from the memory.

        - batch_size: The number of experiences in the sample.
        - The method returns batches of states, actions, rewards, next_states, and done flags,
          converting them to PyTorch tensors and transferring them to the appropriate device (GPU or CPU).
        - The data is extracted from the memory and stacked into numpy arrays before being converted
          into PyTorch tensors (using `torch.from_numpy`).
        - `states`, `next_states`, and `rewards` are floating-point tensors, `actions` are long-type (integer),
          and `dones` is a floating-point tensor representing whether the episode has ended (1.0 for done, 0.0 for not done).
    """

'\n    A class to implement a replay memory buffer for experience replay in reinforcement learning.\n\n    __init__(self, capacity):\n        Initialises the replay memory with a specific capacity.\n        \n        - capacity: Maximum number of experiences to store in memory.\n        - The device is set to GPU if available, otherwise, it defaults to CPU.\n        - An empty list of `memories` is initialised to store the experiences.\n\n    push(self, event):\n        Adds an experience to the memory and ensures the memory size stays within the capacity limit.\n        \n        - event: A tuple representing an experience (state, action, reward, next_state, done).\n        - If the memory exceeds the specified capacity, the oldest experience is removed.\n\n    sample(self, batch_size):\n        Samples a random batch of experiences from the memory.\n        \n        - batch_size: The number of experiences in the sample.\n        - The method returns batches of states, actions, rewar

In [8]:
class Agent():

  def __init__(self, state_size, action_size):
    """
        Initializes the agent with a Q-learning network, optimizer, and replay memory.

        - state_size: The size of the state space.
        - action_size: The size of the action space.
        - local_qnetwork: The local Q-network for estimating the Q-values.
        - target_qnetwork: The target Q-network for stable updates.
        - optimizer: The Adam optimizer used to optimize the Q-network's parameters.
        - memory: A replay memory to store experiences.
        - t_step: A counter for controlling the frequency of learning.
        """
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.state_size = state_size
    self.action_size = action_size
    self.local_qnetwork = Network(state_size, action_size).to(self.device)
    self.target_qnetwork = Network(state_size, action_size).to(self.device)
    self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr = learning_rate)
    self.memory = ReplayMemory(replay_buffer_size)
    self.t_step = 0

  def step(self, state, action, reward, next_state, done):
    """
        Stores an experience in the memory and learns every 4 steps.

        - state: The current state of the environment.
        - action: The action taken by the agent.
        - reward: The reward received after performing the action.
        - next_state: The next state after performing the action.
        - done: A boolean indicating if the episode has ended.
        """

    self.memory.push((state, action, reward, next_state, done))
    self.t_step = (self.t_step + 1) % 4
    if self.t_step == 0:
      if len(self.memory.memory) > minibatch_size:
        experiences = self.memory.sample(100)
        self.learn(experiences, discount_factor)

  def act(self, state, epsilon = 0.):
    """
        Returns the action selected by the agent, either using epsilon-greedy strategy or randomly.

        - state: The current state of the environment.
        - epsilon: The epsilon value for controlling exploration vs exploitation.

        If random.random() > epsilon, selects the best action (greedy).
        Otherwise, selects a random action (exploration).
        """

    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
    self.local_qnetwork.eval()
    with torch.no_grad():
      action_values = self.local_qnetwork(state)
    self.local_qnetwork.train()
    if random.random() > epsilon:
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))

  def learn(self, experiences, discount_factor):
    """
        Updates the local Q-network using the experiences and the Bellman equation.

        - experiences: A batch of experiences sampled from the memory.
        - discount_factor: The discount factor (gamma) used in the Bellman equation.
        """
    states, next_states, actions, rewards, dones = experiences
    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    q_targets = rewards + discount_factor * next_q_targets * (1 - dones)
    q_expected = self.local_qnetwork(states).gather(1, actions)
    loss = F.mse_loss(q_expected, q_targets)
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()
    self.soft_update(self.local_qnetwork, self.target_qnetwork, interpolation_parameter)

  def soft_update(self, local_model, target_model, interpolation_parameter):
    """
        Performs a soft update of the target network's parameters based on the local network's parameters.

        - local_model: The current Q-network.
        - target_model: The target Q-network.
        - interpolation_parameter: The interpolation factor for the soft update.
        """
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)

In [9]:
agent=Agent(state_size,number_action)

In [10]:
number_episodes=2000
maximum_number_timesteps_per_episode=1000
epsilon_starting_value=1.0
epsilon_ending_value=0.01
epsilon_decay_value=0.995
epsilon=epsilon_starting_value
scores_on_100_episodes=deque(maxlen=100)
for episode in range(1, number_episodes + 1):
  state, _ = env.reset()
  score = 0
  for t in range(maximum_number_timesteps_per_episode):
    action = agent.act(state, epsilon)
    next_state, reward, done, _, _ = env.step(action)
    agent.step(state, action, reward, next_state, done)
    state = next_state
    score += reward
    if done:
      break
  scores_on_100_episodes.append(score)
  epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end = "")
  if episode % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
  if np.mean(scores_on_100_episodes) >= 200.0:
    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode - 100, np.mean(scores_on_100_episodes)))
    torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
    break

Episode 100	Average Score: -158.10
Episode 200	Average Score: -86.17
Episode 300	Average Score: -20.13
Episode 400	Average Score: 17.03
Episode 500	Average Score: 79.41
Episode 600	Average Score: 164.79
Episode 680	Average Score: 200.07
Environment solved in 580 episodes!	Average Score: 200.07


In [13]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v3')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()